<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CST3512/blob/main/CST3512_CLXX_Mapping_co2_emmissions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CST3512 Class  
**Mapping co2 Data from OWID**     

Read a `.csv` file `annual-co2-emissions-per-country.csv` from the current working directory and map emissions data by country.

 - Read the data into a dataframe  

 - Ask the user which range of years to map (default to all years)    

 - Present a Choropleth map of the type of CO2 emission by country (ISO code) for the seleced range of years


*this notebook reads a .csv file which distilled data from the OWID file available [online](https://github.com/owid/co2-data).*

*the source file read: [annual-co2-emissions-per-country.csv](https://drive.google.com/file/d/1BzgiANIXNqRHnU85vzB3gHTGBesHuz4F/view?usp=sharing)*    


---

## Houskeeping / Imports

Import the usual suspects. 

In [78]:
#import Pandas 
import pandas as pd

#import Numpy
import numpy as np 

Install and import `geojson` for map data files.

In [ ]:
# install geojson
!pip install geojson

In [80]:
# import geojson
import geojson 

Install and import `geopandas`    

In [ ]:
# install geopandas
!pip install geopandas

In [82]:
# import geopandas and folium 
import geopandas as gpd 

Import `folium` and desired `folium` objects    

In [83]:
import folium
from folium import plugins
from folium.features import GeoJsonTooltip

Import `matplotlib`    

In [84]:
import matplotlib

Intall and import a utility function for producing Jenks Natural Breaks     

In [ ]:
!pip install jenkspy

In [86]:
import jenkspy

Optional: Import **requests**, if desired    

In [88]:
import requests

Optional: Import **JSON** library, if desired    

In [87]:
import json

---

## Define Functions 

This is a PLACEHOLDER

In [ ]:
###PLACEHOLDER###

---

## Source Data 

**This notebook reads data files directly from web resources (URLs).**     

*the source CO2 emissions file read: [annual-co2-emissions-per-country.csv](https://drive.google.com/file/d/1BzgiANIXNqRHnU85vzB3gHTGBesHuz4F/view?usp=sharing)*     


###Source Information Dataset 

In [ ]:
# the following code reads the `annual-co2-emissions-per-country.csv` file into an `emits_df` dataframe
url='https://drive.google.com/file/d/1BzgiANIXNqRHnU85vzB3gHTGBesHuz4F/view?usp=sharing'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
emits_df = pd.read_csv(dwn_url)
print(emits_df.head())

###Source Geo Dataset   

Option 1: read the `geojson` directly from a **URL**    

In [90]:
# the following may be throttled by the server 
country_url  = "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_50m_admin_0_countries.geojson" 
country_json = json.loads(requests.get(country_url).text)

Option 2: make of copy of the **URL** file in the current working directory, then read that file

In [ ]:
!curl "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_50m_admin_0_countries.geojson" -o world_50m_countries.geojson

In [92]:
country_geo_data = "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_50m_admin_0_countries.geojson"

The **natural Earth Vector**  `geojson` file (`ne_50m_admin_0_countries.geojson`) includes the following features: 

   - **properties** - contains a list of **features** which are properties of the map shape (a country) such as the short, long, and formal versions of the country's name and other country-level properties, including:      

       - **`ISO_A3`** - as a three-alpha string with the ISO Code for the country    
       - **`NAME`** - as an alpha string with the short name for the country    




*To specify the <b>ISO Code</b> property of countries in this `geojson` dataset, use the following notation:*    
```
feature.properties.ISO_A3    
```    

*To specify the <b>Short Name</b> property of countries in this `geojson` dataset, use the following notation:*    
```
feature.properties.NAME    
```    



In [110]:
with open('/content/world_50m_countries.geojson') as j:
    country_geo_json = json.load(j)

## Wrangling and Transformation    

Rename the **ISO Code** column name from `Code` to `ISO_A3` in the CO2 emissions dataframe `emits_df`      

In [93]:
emits_df.rename(columns = {'Code':'ISO_A3'}, inplace = True)

Rename the **CO2 emissions** column name from `Annual CO2 emissions` to `co2_emissions` in the CO2 emissions dataframe `emits_df`      

In [94]:
emits_df.rename(columns = {'Annual CO2 emissions':'co2_emissions'}, inplace = True)

---

###Create Base Map for World

Create a base `world_map` with the NYC United Nations Building at the center

Creating a base map with the default **Folium** map    

In [125]:
# initialize a base layer map and store it in a `world_map` object
# ... using the `cartodbpositron` tile set for a light background to the map
world_map = folium.Map(location = [40.742997028, -73.96749613], tiles='cartodbpositron', zoom_start = 15)    

In [ ]:
# Optional - view the `world_map` base map
world_map

Add the country `geojson` data with **ISO Codes** to `world_map`    

In [ ]:
world_map.add_child(folium.GeoJson("/content/world_50m_countries.geojson", name="geojson").add_to(world_map))

# folium.LayerControl().add_to(world_map) 

In [ ]:
world_map

---

##Add CO2 Emissions Choropleth To The Map

Option A: Creating Choropleth object and adding it to the map with `LayerControl`    

In [ ]:
folium.Choropleth(
    # geographical locations
    geo_data = country_geo_data,                     
    name = "CO2_emissions",
    
    # the data set we are using
    data = emits_df,                        
    columns = ["ISO_A3", "co2_emissions"],   
    # YlGn refers to yellow and green
    fill_color = "YlGn",                      
    fill_opacity = 0.7,
    line_opacity = .1,
    key_on = "feature.properties.ISO_A3",
    legend_name="CO2 Emissions",
    bins = 5
).add_to(world_map)   

folium.LayerControl().add_to(world_map) 

Option B: Creating Choropleth object and adding it as a **child** to the map with `add_child()`  

In [ ]:
world_map.add_child(folium.Choropleth(
    # geographical locations
    geo_data = country_geo_data,                     
    name = "CO2_emissions",
    
    # the data set we are using
    data = emits_df,                        
    columns = ["ISO_A3", "co2_emissions"],  

    # YlGn refers to yellow and green
    fill_color = "YlGn",                      
    fill_opacity = 0.7,
    line_opacity = .1,

    # default color for missing data
    nan_fill_color="white",
    nan_fill_opacity= 0.5,

    # key mapping field (maps to columns[0])
    key_on = "feature.properties.ISO_A3",

    # reference attributes for information mapped
    bins=5,
    legend_name="CO2 Emissions",
   ))

In [ ]:
world_map

---

## Write Results to .HTML File(s)       

In [127]:
# write the world_map HTML to `world_co2_map.html`
world_map.save('world_co2_map.html')

---

#  

#Additional Resources    

  - [Mapping in Python: Folium by Robsky Huntley on VIA Learning Lab](https://vialab.mit.edu/tutorials/module/mapping-in-python-folium/) for an exercise which maps tree density in Cambridge, MA and the surrouding area.     

  - [Stamen Toner ,Stamen Terrain and Mapbox Bright Maps in Python-Folium](https://www.geeksforgeeks.org/stamen-toner-stamen-terrain-and-mapbox-bright-maps-in-python-folium/) examples of using different built-in map tiles in **Folium** from Geeks for Geeks 

  - ['Folium Map Tiles' by DeParkes](https://deparkes.co.uk/2016/06/10/folium-map-tiles/) more examples of map tiles available in **Folium**   

  - ['Create and Visualize Choropleth Maps with Folium' by Analytics Vidhya on Medium](https://medium.com/analytics-vidhya/create-and-visualize-choropleth-map-with-folium-269d3fd12fa0) for examples of different approaches to color coding map elements as well as more formatting and style options    



---

